# DaGMM with VAE, kdd99cup

- Full network with VAE instead of AE

In [1]:
from keras.layers import Input, Dense, Activation, Lambda, Dropout, Concatenate, Reshape
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard
from keras import optimizers
from keras import backend as K

from keras.regularizers import l2


import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)
tf.set_random_seed(0)

/home/gabrielle/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


- ### Timestamp for experiments

In [2]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M")
print(timestamp, "")

2018-07-27_17:21 


In [3]:
### load dataset

class KDD99Dataset:
    def __init__(self, filename_train, filename_test):
        with np.load(filename_train) as data: 
            self.x_train = data["x_train"]
            self.y_train = data["y_train"]
            
        with np.load(filename_test) as data: 
            self.x_test = data["x_test"]
            #self.y_test = data["y_test"]
            
            
    def train_num_examples(self):
        return len(self.x_train)

    def train_next_batch(self,batch_size):
        choices = np.random.choice(len(self.x_train), size=batch_size, replace=False)

        batch_x_train = self.x_train[choices]
        batch_y_train = self.y_train[choices]

        batch_train = (batch_x_train, batch_y_train)

        return batch_train
    
    def test_next_batch(self,batch_size):
        choices = np.random.choice(len(self.x_test), size=batch_size, replace=False)

        batch_x_test = self.x_test[choices]
        #batch_y_train = y_train[choices]

        batch_train = (batch_x_test, None) #batch_y_train)

        return batch_train


In [4]:
kdd99 = KDD99Dataset("../datasets/kddcup/kdd99_train-randomState_None.npz", 
                     "../datasets/kddcup/kdd99_test-randomState_None.npz")

n_samples = kdd99.train_num_examples()

In [5]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)

Based on this, we define now a class "VariationalAutoencoder" with a [sklearn](http://scikit-learn.org)-like interface that can be trained incrementally with mini-batches using partial_fit. The trained model can be used to reconstruct unseen input, to generate new samples, and to map inputs to the latent space.

In [6]:
class VariationalAutoencoder(object):
    """ Variation Autoencoder (VAE) with an sklearn-like interface implemented using TensorFlow.
    
    This implementation uses probabilistic encoders and decoders using Gaussian 
    distributions and  realized by multi-layer perceptrons. The VAE can be learned
    end-to-end.
    
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, 
                 transfer_fct=tf.nn.softplus, 
                 learning_rate=0.0001, 
                 batch_size=1024,
                 gmm_mixtures=4,  ###default for the kdd99, from paper
                 lambda_1 = 0.1,
                 lambda_2 = 0.005,
                ):
        
        
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.gmm_mixtures = gmm_mixtures
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2
        
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        
        # Create autoencoder network
        self._create_network()
        
        ### init gmm parameters
        self.phi, self.mu, self.sigma = self._init_gmm()
        
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        ###self._create_loss_optimizer()
        self._total_loss()  ### new one
        
        # Initializing the tensor flow variables
        init = tf.global_variables_initializer()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        network_weights = self._initialize_weights(**self.network_architecture)

        # Use recognition network to determine mean and 
        # (log) variance of Gaussian distribution in latent
        # space
        self.z_mean, self.z_log_sigma_sq = \
            self._recognition_network(network_weights["weights_recog"], 
                                      network_weights["biases_recog"])

        # Draw one sample z from Gaussian distribution
        n_z = self.network_architecture["n_z"]
        eps = tf.random_normal((self.batch_size, n_z), 0, 1, 
                               dtype=tf.float32)
        # z = mu + sigma*epsilon
        self.z = tf.add(self.z_mean, 
                        tf.multiply(tf.sqrt(tf.exp(self.z_log_sigma_sq)), eps))

        # Use generator to determine mean of
        # Bernoulli distribution of reconstructed input
        self.x_reconstr_mean = \
            self._generator_network(network_weights["weights_gener"],
                                    network_weights["biases_gener"])
        
        ## cosine similarity
        self.cossim = self._layer_cossim(self.x, self.x_reconstr_mean)
        
        ## relative euc similarity
        self.relative_euc = self._layer_relative_eucdist(self.x, self.x_reconstr_mean)
        
        ## concat layers (z in original paper)
        self.concat = self._layer_concat(self.z, self.cossim, self.relative_euc)
        
        
        ##### estimation net
        input_est = self.concat 
        
        est_layer = Dense(10, activation='tanh', kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001))(input_est)   ####(input_est)
        est_layer = Dropout(0.5)(est_layer)
        self.est_output = Dense(self.gmm_mixtures, activation='softmax', kernel_regularizer=l2(0.00001), bias_regularizer=l2(0.00001), name='gamma')(est_layer)

        
    def _layer_cossim(self, input_layer, output_layer):
        '''
        a: batch x 120
        b: batch x 120 

        output: batch x 1
        '''
        a, b = input_layer, output_layer  #a_b
    
        norm_a = K.sqrt(K.sum(a ** 2, axis=-1))
        norm_b = K.sqrt(K.sum(b ** 2, axis=-1))
    
        out = K.sum(a * b, axis=-1) / (norm_a * norm_b)
        out = K.reshape(out, [self.batch_size, 1])
        
        return out
        
        
    def _layer_relative_eucdist(self, input_layer, output_layer):
        '''
        a: batch x 120
        b: batch x 120 

        output: batch x 1
        '''
        a, b = input_layer, output_layer  #a_b

        norm_diff = K.sqrt(K.sum((a - b)**2, axis=-1))
        norm_a = K.sqrt(K.sum(a ** 2, axis=-1))

        out = norm_diff / norm_a
        out = K.reshape(out, [self.batch_size, 1])

        return out
        
        
    def _layer_concat(self, lowdim, cossim, relative_euc):
        return K.concatenate([lowdim, cossim, relative_euc])
    
    
    def _init_gmm(self):
        k = self.gmm_mixtures
        self.d = d = int(self.concat.get_shape()[1])
        
        phi = tf.get_variable("phi",
                      shape=(k),
                      dtype=tf.float32,
                      initializer=tf.zeros_initializer(),
                      trainable=False)

        mu = tf.get_variable("mu",
                              shape=(k,d),
                              dtype=tf.float32,
                              initializer=tf.zeros_initializer(), 
                              trainable=False)

        sigma_init = np.repeat([np.eye(d, dtype=np.float32)], k, axis=0)

        sigma = tf.get_variable("sigma",
                              shape=(k,d,d),
                              dtype=tf.float32,
                              initializer=tf.constant_initializer(sigma_init),
                              trainable=False)
        
        return (phi, mu, sigma)
        
    
    def _compute_energy(self, z_i):
        """
        compute E(z_i) in loss function
        """

        #inside_sum = 0
        inside_sum = tf.zeros(()) 
        for cluster in range(self.gmm_mixtures):
            diff = tf.reshape(z_i - self.mu[cluster], (1,-1))   ### (1,3)
            diff_t = tf.reshape(diff, (-1,1)) #diff.reshape(-1,1)   ### (3,1)

            sigma_inv = tf.linalg.inv(self.sigma[cluster]) ### (3,3)

            exp_term = tf.exp(-0.5 * tf.matmul(diff, tf.matmul(sigma_inv, diff_t)))    ### (1,1)

            denom = tf.sqrt(tf.linalg.det(2 * np.pi * self.sigma[cluster]))

            inside_sum += self.phi[cluster] * (exp_term / denom)   ### (1,1)


        inside_sum = tf.reshape(inside_sum, ())
        sample_energy = -tf.log(inside_sum + 1e-6, name="sample_energy")

        ### flatten inside_sum and return log of it
        return sample_energy
    
    
    def save_weights(self, filename = "./" + timestamp + "-tfsave"):  
        ### save model weights
        saver = tf.train.Saver()
        saver.save(self.sess, filename)
        print("saved to:",filename)
        
        
        
    def load_weights(self, filename = "./" + timestamp + "-tfsave.meta"):
        ### load weights for test notebook
        saver = tf.train.import_meta_graph(filename)
        saver.restore(self.sess, tf.train.latest_checkpoint("./"))
    
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  n_hidden_gener_2, 
                            n_input, n_z):
        all_weights = dict()
        all_weights['weights_recog'] = {
            'h1': tf.Variable(xavier_init(n_input, n_hidden_recog_1)),
            'h2': tf.Variable(xavier_init(n_hidden_recog_1, n_hidden_recog_2)),
            'out_mean': tf.Variable(xavier_init(n_hidden_recog_2, n_z)),
            'out_log_sigma': tf.Variable(xavier_init(n_hidden_recog_2, n_z))}
        all_weights['biases_recog'] = {
            'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
            'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
            'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
            'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32))}
        all_weights['weights_gener'] = {
            'h1': tf.Variable(xavier_init(n_z, n_hidden_gener_1)),
            'h2': tf.Variable(xavier_init(n_hidden_gener_1, n_hidden_gener_2)),
            'out_mean': tf.Variable(xavier_init(n_hidden_gener_2, n_input)),
            'out_log_sigma': tf.Variable(xavier_init(n_hidden_gener_2, n_input))}
        all_weights['biases_gener'] = {
            'b1': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32)),
            'b2': tf.Variable(tf.zeros([n_hidden_gener_2], dtype=tf.float32)),
            'out_mean': tf.Variable(tf.zeros([n_input], dtype=tf.float32)),
            'out_log_sigma': tf.Variable(tf.zeros([n_input], dtype=tf.float32))}
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network), which
        # maps inputs onto a normal distribution in latent space.
        # The transformation is parametrized and can be learned.
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        z_mean = tf.add(tf.matmul(layer_2, weights['out_mean']),
                        biases['out_mean'])
        z_log_sigma_sq = \
            tf.add(tf.matmul(layer_2, weights['out_log_sigma']), 
                   biases['out_log_sigma'])
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self, weights, biases):
        # Generate probabilistic decoder (decoder network), which
        # maps points in latent space onto a Bernoulli distribution in data space.
        # The transformation is parametrized and can be learned.
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.z, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        x_reconstr_mean = \
            tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['out_mean']), 
                                 biases['out_mean']))
        return x_reconstr_mean

        
    def _update_gmm(self):
        gamma = self.est_output ### output of estimation net (batch size x 4)
        z = self.concat ### concat output (batch size x 3)
        
        
        ### get parameters
        self.updates_gmm = []
        phi = self.phi
        mu = self.mu
        sigma = self.sigma
        k = self.gmm_mixtures
        d = self.d
        N = self.batch_size

        ################### phi #################
        update_phi = tf.assign(phi, 
                               tf.reduce_sum(gamma/self.batch_size, axis=0), 
                               name="update_phi")
        self.updates_gmm.append(update_phi)
        #######################################

        ################## mu ################
        for cluster in range(k):
            ### get the corresponding column of predictions
            gamma_cluster = tf.reshape(gamma[:,cluster], (-1,1))    ### (N x 1)

            ### duplicate column d times
            gamma_cluster_tile = tf.tile(gamma_cluster, (1,d))   ### (N x d)

            ### sum over all batch and divide
            matmul = tf.matmul(z, gamma_cluster_tile, transpose_a=True)   ### (dxd)

            result = tf.reduce_sum(matmul, axis=0, name="mu_matmul_red") / tf.reduce_sum(gamma[:,cluster], axis = 0, name="mu_gamma_red")

            update_mu = tf.assign(mu[cluster], 
                                  result,
                                  name="update_mu"+str(cluster))
            self.updates_gmm.append(update_mu)
        #########################################

        ################ sigma ###############
        for cluster in range(k):
            ### expand gamma for each sample
            gamma_cluster = tf.reshape(gamma[:,cluster], (-1,1))    ### (N x 1)
            gamma_cluster_expand = tf.expand_dims(gamma_cluster, 1) ### (N x 1 x 1)


            #### calculating diff
            ## expand mu and z
            ######## TODO: race condition??? (get mu after its updated value (auto??))
            with tf.control_dependencies(self.updates_gmm):
                mu_cluster = tf.reshape(mu[cluster], (1,-1))   ### (1 x d)

            mu_expand = tf.expand_dims(mu_cluster, 1)      ### (1 x 1 x 3)
            mu_expand_tile = tf.tile(mu_expand, tf.stack([N, 1, 1]))   ### (N x 1 x d)

            z_expand = tf.expand_dims(z, 1)    ### (N x 1 x d)

            diff = z_expand - mu_expand_tile   ### (N x 1 x d)

            ### matmul in the upper part
            matmul = tf.matmul(diff, diff, transpose_a=True)  ### (N x d x d)


            ### nominator
            nom = gamma_cluster_expand * matmul   ### (N x d x d)
            nom_reduced = tf.reduce_sum(nom, axis=0) ### (d x d)


            ### denominator
            denom = tf.reduce_sum(gamma_cluster)  ### single value, zero dim

            update_sigma = tf.assign(sigma[cluster], 
                                     nom_reduced/denom, 
                                     name ="update_sigma"+str(cluster))
            self.updates_gmm.append(update_sigma)
            
            
             
    def _create_loss_optimizer(self):
        # The loss is composed of two terms:
        # 1.) The reconstruction loss (the negative log probability
        #     of the input under the reconstructed Bernoulli distribution 
        #     induced by the decoder in the data space).
        #     This can be interpreted as the number of "nats" required
        #     for reconstructing the input when the activation in latent
        #     is given.
        # Adding 1e-10 to avoid evaluation of log(0.0)
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(1e-6 + self.x_reconstr_mean)
                           + (1-self.x) * tf.log(1e-6 + 1 - self.x_reconstr_mean),
                           1)
        # 2.) The latent loss, which is defined as the Kullback Leibler divergence 
        ##    between the distribution in latent space induced by the encoder on 
        #     the data and some prior. This acts as a kind of regularizer.
        #     This can be interpreted as the number of "nats" required
        #     for transmitting the the latent space distribution given
        #     the prior.
        latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
                                           - tf.square(self.z_mean) 
                                           - tf.exp(self.z_log_sigma_sq), 1)
        self.cost = tf.reduce_mean(reconstr_loss + latent_loss)   # average over batch
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdagradOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
               
        
    def _total_loss(self):
        ### reconstr loss + latent loss + energy + reg term for sigma
        
        ### reconstr loss
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(1e-6 + self.x_reconstr_mean)
                           + (1-self.x) * tf.log(1e-6 + 1 - self.x_reconstr_mean),
                           1)

        ### latent loss
        latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
                                           - tf.square(self.z_mean) 
                                           - tf.exp(self.z_log_sigma_sq), 1)
        
        ### update gmm params for energy
        self._update_gmm()
        
        
        ### energy
        ### dependency control
        z = self.concat
        
        with tf.control_dependencies(self.updates_gmm):  
            ### sample energy   
            sample_en_batch = tf.map_fn(lambda z_i: self._compute_energy(z_i), z)
            sample_en = tf.reduce_mean(sample_en_batch, axis=0)
            sample_en *= self.lambda_1

            ### diagonal regularization
            p = tf.reduce_sum(1 / tf.matrix_diag_part(self.sigma))
            p *= self.lambda_2
            
        ### vae loss instead of ae loss    
        vae_loss = tf.reduce_mean(reconstr_loss + latent_loss)
        
        ### total loss
        self.cost = vae_loss + sample_en + p
        
         # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
    
    def saveGmmParams(self, filename = timestamp + "-gmmParams.npz"):
        phi_ = K.eval(self.phi)
        mu_ = K.eval(self.mu)
        sigma_ = K.eval(self.sigma)
        np.savez_compressed(filename, phi=phi_, mu=mu_, sigma=sigma_)
        print("saved to:", filename)
        
    def printGmmParams(self):
        print("#### GMM params ####")
        print("phi:\n",K.eval(self.phi),"\n")
        print("mu:\n",K.eval(self.mu),"\n")
        print("sigma:\n", K.eval(self.sigma),"\n")
        

    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
        opt, cost = self.sess.run((self.optimizer, self.cost), 
                                  feed_dict={self.x: X})
        return cost
    
    def transform(self, X):
        """Transform data by mapping it into the latent space."""
        # Note: This maps to mean of distribution, we could alternatively
        # sample from Gaussian distribution
        mu = self.sess.run(self.z_mean, feed_dict={self.x: X})
        sigma = self.sess.run(self.z_log_sigma_sq, feed_dict={self.x: X}) ###!!! (doğru mu/gerekli mi??)
        return mu, sigma
    
    def generate(self, z_mu=None):
        """ Generate data by sampling from latent space.
        
        If z_mu is not None, data for this point in latent space is
        generated. Otherwise, z_mu is drawn from prior in latent 
        space.        
        """
        if z_mu is None:
            z_mu = np.random.normal(size=self.network_architecture["n_z"])
        # Note: This maps to mean of distribution, we could alternatively
        # sample from Gaussian distribution
        return self.sess.run(self.x_reconstr_mean, 
                             feed_dict={self.z: z_mu})
    
    def reconstruct(self, X):
        """ Use VAE to reconstruct given data. """
        return self.sess.run(self.x_reconstr_mean, 
                             feed_dict={self.x: X})

In general, implementing a VAE in tensorflow is relatively straightforward (in particular since we don not need to code the gradient computation). A bit confusing is potentially that all the logic happens at initialization of the class (where the graph is generated), while the actual sklearn interface methods are very simple one-liners.

We can now define a simple fuction which trains the VAE using mini-batches:

In [7]:
import time

def train(network_architecture, 
          learning_rate=0.0001,
          batch_size=1024, 
          training_epochs=70, 
          display_step=1):
    
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    history_loss = dict()
    

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples / batch_size)
        # Loop over all batches
        
        loss_batch = []
        
        start = time.time()
        for i in range(total_batch):
            batch_xs, _ = kdd99.train_next_batch(batch_size) #mnist.train.next_batch(batch_size)

            # Fit training using batch data
            cost = vae.partial_fit(batch_xs)
            
            loss_batch.append(cost)
            
            print(" batch:",i,"cost:",cost)
            # Compute average loss
            avg_cost += cost / n_samples * batch_size

        end = time.time()
        
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("--- end of epoch:", '%04d' % (epoch+1), 
                  "avg cost=", "{:.9f}".format(avg_cost),
                  "time=", "{:.3f} seconds".format(end-start), )
            
        ### append loss to history
        history_loss[str(epoch)] = loss_batch
        history_loss[str(epoch)+"-avg"] = avg_cost
            
    return vae, history_loss

### Train network

In [8]:

network_architecture = \
    dict(n_hidden_recog_1=60, # 1st layer encoder neurons
         n_hidden_recog_2=30, # 2nd layer encoder neurons
         n_hidden_gener_1=30, # 1st layer decoder neurons
         n_hidden_gener_2=60, # 2nd layer decoder neurons
         n_input=120,  # kdd99 data input dimension
         n_z=1,
        )  # dimensionality of latent space

vae, history_loss = train(network_architecture, 
            training_epochs=60, 
            display_step=1, 
            batch_size=1024)

 batch: 0 cost: 87.906975
 batch: 1 cost: 88.05488
 batch: 2 cost: 86.931114
 batch: 3 cost: 86.64176
 batch: 4 cost: 86.41183
 batch: 5 cost: 86.03902
 batch: 6 cost: 85.81755
 batch: 7 cost: 85.43244
 batch: 8 cost: 85.06063
 batch: 9 cost: 84.96531
 batch: 10 cost: 84.45251
 batch: 11 cost: 84.38164
 batch: 12 cost: 83.89177
 batch: 13 cost: 83.614395
 batch: 14 cost: 83.35261
 batch: 15 cost: 82.965645
 batch: 16 cost: 82.848854
 batch: 17 cost: 82.404785
 batch: 18 cost: 82.057205
 batch: 19 cost: 82.07605
 batch: 20 cost: 81.613846
 batch: 21 cost: 81.43624
 batch: 22 cost: 81.26042
 batch: 23 cost: 80.84142
 batch: 24 cost: 80.6421
 batch: 25 cost: 80.41416
 batch: 26 cost: 80.19581
 batch: 27 cost: 79.88342
 batch: 28 cost: 79.634026
 batch: 29 cost: 79.48731
 batch: 30 cost: 79.12823
 batch: 31 cost: 78.953735
 batch: 32 cost: 78.56942
 batch: 33 cost: 78.40615
 batch: 34 cost: 78.2075
 batch: 35 cost: 78.08491
 batch: 36 cost: 77.81963
 batch: 37 cost: 77.428024
 batch: 38 co

 batch: 112 cost: 30.302824
 batch: 113 cost: 29.745079
 batch: 114 cost: 30.130095
 batch: 115 cost: 29.99985
 batch: 116 cost: 29.592545
 batch: 117 cost: 29.368376
 batch: 118 cost: 29.689518
 batch: 119 cost: 29.028807
 batch: 120 cost: 29.145739
 batch: 121 cost: 29.36061
 batch: 122 cost: 29.43077
 batch: 123 cost: 28.346457
 batch: 124 cost: 28.905066
 batch: 125 cost: 28.874445
 batch: 126 cost: 29.017132
 batch: 127 cost: 28.13932
 batch: 128 cost: 28.183174
 batch: 129 cost: 28.502485
 batch: 130 cost: 28.495914
 batch: 131 cost: 27.670624
 batch: 132 cost: 28.069693
 batch: 133 cost: 28.060009
 batch: 134 cost: 27.950642
 batch: 135 cost: 27.9091
 batch: 136 cost: 27.483912
 batch: 137 cost: 27.625542
 batch: 138 cost: 27.492474
 batch: 139 cost: 27.393093
 batch: 140 cost: 27.482403
 batch: 141 cost: 27.381224
 batch: 142 cost: 27.21525
 batch: 143 cost: 26.981213
 batch: 144 cost: 26.595896
 batch: 145 cost: 27.052486
 batch: 146 cost: 26.75457
 batch: 147 cost: 26.763386


 batch: 22 cost: 14.905379
 batch: 23 cost: 14.909476
 batch: 24 cost: 14.694915
 batch: 25 cost: 15.140731
 batch: 26 cost: 14.453045
 batch: 27 cost: 14.870052
 batch: 28 cost: 15.284013
 batch: 29 cost: 14.967293
 batch: 30 cost: 14.663746
 batch: 31 cost: 14.681349
 batch: 32 cost: 14.62724
 batch: 33 cost: 14.852444
 batch: 34 cost: 14.3403
 batch: 35 cost: 14.591255
 batch: 36 cost: 14.980503
 batch: 37 cost: 15.133198
 batch: 38 cost: 14.680979
 batch: 39 cost: 15.013018
 batch: 40 cost: 14.645435
 batch: 41 cost: 14.739039
 batch: 42 cost: 14.386194
 batch: 43 cost: 14.499825
 batch: 44 cost: 14.5132265
 batch: 45 cost: 14.651706
 batch: 46 cost: 14.6049
 batch: 47 cost: 14.768967
 batch: 48 cost: 14.4001
 batch: 49 cost: 14.755332
 batch: 50 cost: 14.141273
 batch: 51 cost: 14.305275
 batch: 52 cost: 14.511115
 batch: 53 cost: 14.593717
 batch: 54 cost: 14.7905855
 batch: 55 cost: 15.027368
 batch: 56 cost: 14.264446
 batch: 57 cost: 14.392962
 batch: 58 cost: 14.472484
 batch

 batch: 127 cost: 11.91623
 batch: 128 cost: 11.97846
 batch: 129 cost: 11.735305
 batch: 130 cost: 11.5961685
 batch: 131 cost: 11.290695
 batch: 132 cost: 11.744042
 batch: 133 cost: 11.792392
 batch: 134 cost: 11.92122
 batch: 135 cost: 11.614195
 batch: 136 cost: 11.955642
 batch: 137 cost: 11.553931
 batch: 138 cost: 11.376132
 batch: 139 cost: 11.510432
 batch: 140 cost: 11.777361
 batch: 141 cost: 11.544035
 batch: 142 cost: 11.477879
 batch: 143 cost: 11.732939
 batch: 144 cost: 11.522572
 batch: 145 cost: 11.23826
 batch: 146 cost: 11.74918
 batch: 147 cost: 11.509448
 batch: 148 cost: 11.602698
 batch: 149 cost: 11.628812
 batch: 150 cost: 11.839922
 batch: 151 cost: 11.106308
 batch: 152 cost: 11.129975
 batch: 153 cost: 11.562373
 batch: 154 cost: 11.185183
 batch: 155 cost: 11.434986
 batch: 156 cost: 11.186495
 batch: 157 cost: 11.240204
 batch: 158 cost: 11.549493
 batch: 159 cost: 11.35001
 batch: 160 cost: 12.106262
 batch: 161 cost: 11.263773
 batch: 162 cost: 11.9347

 batch: 39 cost: 8.794281
 batch: 40 cost: 9.080266
 batch: 41 cost: 9.277722
 batch: 42 cost: 8.740868
 batch: 43 cost: 9.076092
 batch: 44 cost: 8.42589
 batch: 45 cost: 8.842435
 batch: 46 cost: 9.16623
 batch: 47 cost: 8.608373
 batch: 48 cost: 9.252191
 batch: 49 cost: 9.40294
 batch: 50 cost: 9.156585
 batch: 51 cost: 8.890187
 batch: 52 cost: 8.751952
 batch: 53 cost: 8.801544
 batch: 54 cost: 9.062824
 batch: 55 cost: 9.083037
 batch: 56 cost: 9.018956
 batch: 57 cost: 8.691755
 batch: 58 cost: 8.566481
 batch: 59 cost: 9.19675
 batch: 60 cost: 8.466992
 batch: 61 cost: 9.051474
 batch: 62 cost: 8.898717
 batch: 63 cost: 8.645118
 batch: 64 cost: 8.78931
 batch: 65 cost: 8.744836
 batch: 66 cost: 8.767022
 batch: 67 cost: 9.003383
 batch: 68 cost: 8.186537
 batch: 69 cost: 8.481422
 batch: 70 cost: 8.654734
 batch: 71 cost: 8.821045
 batch: 72 cost: 8.786351
 batch: 73 cost: 8.713124
 batch: 74 cost: 8.918275
 batch: 75 cost: 8.527761
 batch: 76 cost: 8.377484
 batch: 77 cost: 

...

 batch: 178 cost: 3.5966454
 batch: 179 cost: 3.7704072
 batch: 180 cost: 3.8030343
 batch: 181 cost: 3.6146588
 batch: 182 cost: 3.932572
 batch: 183 cost: 3.1541443
 batch: 184 cost: 3.900742
 batch: 185 cost: 3.1932974
 batch: 186 cost: 3.2713165
 batch: 187 cost: 3.5121372
 batch: 188 cost: 3.2873569
 batch: 189 cost: 3.2473485
 batch: 190 cost: 3.4980195
 batch: 191 cost: 3.4876363
 batch: 192 cost: 3.809979
--- end of epoch: 0053 avg cost= 3.533388074 time= 111.022 seconds
 batch: 0 cost: 3.154743
 batch: 1 cost: 3.2212038
 batch: 2 cost: 3.416369
 batch: 3 cost: 3.444234
 batch: 4 cost: 3.493559
 batch: 5 cost: 4.378826
 batch: 6 cost: 4.362266
 batch: 7 cost: 3.3401403
 batch: 8 cost: 3.632577
 batch: 9 cost: 4.4119897
 batch: 10 cost: 3.5042303
 batch: 11 cost: 3.5080237
 batch: 12 cost: 3.2373116
 batch: 13 cost: 3.2425013
 batch: 14 cost: 3.3072176
 batch: 15 cost: 3.874988
 batch: 16 cost: 4.1678414
 batch: 17 cost: 3.4894168
 batch: 18 cost: 3.208928
 batch: 19 cost: 4.030

 batch: 91 cost: 3.9964337
 batch: 92 cost: 3.3747911
 batch: 93 cost: 3.7557325
 batch: 94 cost: 4.4008636
 batch: 95 cost: 3.5381036
 batch: 96 cost: 3.5225797
 batch: 97 cost: 4.3409777
 batch: 98 cost: 3.3630922
 batch: 99 cost: 3.3232646
 batch: 100 cost: 3.0826373
 batch: 101 cost: 3.822284
 batch: 102 cost: 3.051478
 batch: 103 cost: 3.2567406
 batch: 104 cost: 3.4770217
 batch: 105 cost: 4.5060863
 batch: 106 cost: 3.4610603
 batch: 107 cost: 2.997371
 batch: 108 cost: 3.5294504
 batch: 109 cost: 3.5093195
 batch: 110 cost: 3.5451355
 batch: 111 cost: 3.614993
 batch: 112 cost: 4.828433
 batch: 113 cost: 3.2048643
 batch: 114 cost: 3.134735
 batch: 115 cost: 3.2478359
 batch: 116 cost: 3.496315
 batch: 117 cost: 2.956808
 batch: 118 cost: 2.9933634
 batch: 119 cost: 3.0229719
 batch: 120 cost: 3.0575619
 batch: 121 cost: 3.9944782
 batch: 122 cost: 4.1124535
 batch: 123 cost: 3.0946786
 batch: 124 cost: 3.461772
 batch: 125 cost: 3.3313198
 batch: 126 cost: 3.2877421
 batch: 12

 batch: 0 cost: 3.38548
 batch: 1 cost: 3.2475584
 batch: 2 cost: 3.3466377
 batch: 3 cost: 3.0487008
 batch: 4 cost: 3.8748693
 batch: 5 cost: 3.71305
 batch: 6 cost: 3.4827218
 batch: 7 cost: 3.355099
 batch: 8 cost: 3.1365614
 batch: 9 cost: 2.993353
 batch: 10 cost: 3.170245
 batch: 11 cost: 3.3582091
 batch: 12 cost: 3.23736
 batch: 13 cost: 3.2249002
 batch: 14 cost: 4.15557
 batch: 15 cost: 4.6157956
 batch: 16 cost: 4.233843
 batch: 17 cost: 3.599718
 batch: 18 cost: 3.0566673
 batch: 19 cost: 3.190795
 batch: 20 cost: 3.100557
 batch: 21 cost: 3.7133703
 batch: 22 cost: 3.1676612
 batch: 23 cost: 3.318298
 batch: 24 cost: 3.004678
 batch: 25 cost: 3.4592643
 batch: 26 cost: 3.3322563
 batch: 27 cost: 3.1348844
 batch: 28 cost: 3.459546
 batch: 29 cost: 3.7727056
 batch: 30 cost: 3.1008046
 batch: 31 cost: 3.5092516
 batch: 32 cost: 3.303688
 batch: 33 cost: 3.1924162
 batch: 34 cost: 3.1168635
 batch: 35 cost: 3.6359425
 batch: 36 cost: 3.0933058
 batch: 37 cost: 2.9806478
 ba

 batch: 110 cost: 3.1106377
 batch: 111 cost: 3.0514643
 batch: 112 cost: 3.0899017
 batch: 113 cost: 3.1002426
 batch: 114 cost: 3.3022604
 batch: 115 cost: 3.3368676
 batch: 116 cost: 3.1189778
 batch: 117 cost: 3.218972
 batch: 118 cost: 3.2647839
 batch: 119 cost: 4.4181633
 batch: 120 cost: 3.0420964
 batch: 121 cost: 3.2522736
 batch: 122 cost: 4.051015
 batch: 123 cost: 3.148604
 batch: 124 cost: 3.119936
 batch: 125 cost: 3.1774955
 batch: 126 cost: 3.0835478
 batch: 127 cost: 3.355116
 batch: 128 cost: 3.545247
 batch: 129 cost: 3.1025982
 batch: 130 cost: 3.0357285
 batch: 131 cost: 3.1200442
 batch: 132 cost: 3.4626017
 batch: 133 cost: 3.2131138
 batch: 134 cost: 2.9504335
 batch: 135 cost: 3.246519
 batch: 136 cost: 3.388592
 batch: 137 cost: 3.3293197
 batch: 138 cost: 3.413599
 batch: 139 cost: 3.2627015
 batch: 140 cost: 2.8727627
 batch: 141 cost: 3.204847
 batch: 142 cost: 3.7680693
 batch: 143 cost: 3.3961377
 batch: 144 cost: 3.1071181
 batch: 145 cost: 3.2854228
 b

 batch: 20 cost: 3.1799753
 batch: 21 cost: 3.8359013
 batch: 22 cost: 3.638718
 batch: 23 cost: 3.2567585
 batch: 24 cost: 3.239408
 batch: 25 cost: 3.4484422
 batch: 26 cost: 3.2030883
 batch: 27 cost: 4.3266706
 batch: 28 cost: 3.4576201
 batch: 29 cost: 4.026409
 batch: 30 cost: 3.14445
 batch: 31 cost: 4.313577
 batch: 32 cost: 4.0497794
 batch: 33 cost: 3.0281868
 batch: 34 cost: 3.2200966
 batch: 35 cost: 3.3137984
 batch: 36 cost: 3.2087018
 batch: 37 cost: 3.3277054
 batch: 38 cost: 2.9738762
 batch: 39 cost: 3.4112413
 batch: 40 cost: 3.4235961
 batch: 41 cost: 3.392781
 batch: 42 cost: 3.4537923
 batch: 43 cost: 3.2194161
 batch: 44 cost: 3.2313771
 batch: 45 cost: 2.961916
 batch: 46 cost: 3.4710674
 batch: 47 cost: 3.3085065
 batch: 48 cost: 2.9666
 batch: 49 cost: 3.2366662
 batch: 50 cost: 3.145366
 batch: 51 cost: 3.0644896
 batch: 52 cost: 3.035055
 batch: 53 cost: 3.328544
 batch: 54 cost: 3.190471
 batch: 55 cost: 3.554823
 batch: 56 cost: 4.302327
 batch: 57 cost: 3

### Save network params

In [9]:
vae.save_weights()

saved to: ./2018-07-27_17:21-tfsave


In [10]:
vae.load_weights(filename = "./" + timestamp + "-tfsave.meta")

INFO:tensorflow:Restoring parameters from ./2018-07-27_17:21-tfsave


### Save loss history

In [11]:
filename = timestamp + "_history"
np.savez_compressed(filename, history_loss=history_loss)

### Save gmm params

In [12]:
vae.saveGmmParams()

saved to: 2018-07-27_17:21-gmmParams.npz
